#### Pulled from : https://www.tensorflow.org/tutorials/quickstart/beginner

## Uses Keras (https://www.tensorflow.org/guide/keras/overview)

1. Load prebuilt dataset
2. Build neural network machine
3. Train neural network
4. Evaluate the machine

## Setup tensorflow

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Using keras sequential (https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

<a href="https://www.tensorflow.org/guide/keras/sequential_model">Sequential</a> is useful for stacking layers where each layer has one input <a href="https://www.tensorflow.org/guide/tensor">tensor</a> and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten">Flatten</a>, <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense">Dense</a>, and <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout">Dropout</a> layers.

For each example, the model returns a vector of <a href="https://developers.google.com/machine-learning/glossary#logits">logits</a> or <a href="https://developers.google.com/machine-learning/glossary#log-odds">log-odds</a> scores, one for each class.

In [8]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.08824034, -0.26079616, -0.07559998,  0.2138725 ,  0.1205364 ,
        -0.33668992, -0.37281877, -0.14891198, -0.19410911,  0.1228098 ]],
      dtype=float32)

The <a href="https://www.tensorflow.org/api_docs/python/tf/nn/softmax">tf.nn.softmax</a> function converts these logits to probabilities for each class: 

In [9]:
tf.nn.softmax(predictions).numpy()

array([[0.09953116, 0.08375659, 0.10079726, 0.13463718, 0.1226393 ,
        0.07763521, 0.07488041, 0.09367198, 0.08953252, 0.12291842]],
      dtype=float32)

Define a loss function for training using <a href="https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy">losses.SparseCategoricalCrossentropy</a>:

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [11]:
loss_fn(y_train[:1], predictions).numpy()

np.float32(2.5557342)

Before you start training, configure and compile the model using Keras <a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile">Model.compile</a>. Set the <a href="https://www.tensorflow.org/api_docs/python/tf/keras/optimizers">optimizer</a> class to adam, set the loss to the loss_fn function you defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### Train and evaluate your model

Use the <a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit">Model.fit</a> method to adjust your model parameters and minimize the loss: 

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2025-03-11 02:56:17.365693: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8593 - loss: 0.4743
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9524 - loss: 0.1549
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9673 - loss: 0.1064
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9732 - loss: 0.0849
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9762 - loss: 0.0742


The <a href="https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate">Model.evaluate</a> method checks the model's performance, usually on a <a href="https://developers.google.com/machine-learning/glossary#validation-set">validation set</a> or <a href="https://developers.google.com/machine-learning/glossary#test-set">test set</a>.

In [14]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - 845us/step - accuracy: 0.9782 - loss: 0.0778


[0.07780635356903076, 0.9782000184059143]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the <a href="https://www.tensorflow.org/tutorials/">TensorFlow tutorials</a>.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [15]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.3448401e-06, 3.9514010e-08, 3.4590385e-06, 6.7980582e-05,
        7.4244300e-10, 3.0320476e-07, 1.0773813e-10, 9.9991727e-01,
        1.5647131e-06, 8.0695418e-06],
       [1.2464202e-08, 9.7346242e-04, 9.9878961e-01, 2.3400983e-04,
        8.3000494e-17, 9.9628562e-07, 8.9176361e-10, 9.0331458e-12,
        1.8857233e-06, 4.2301134e-14],
       [1.6381179e-08, 9.9951446e-01, 3.7102227e-06, 6.1771652e-06,
        1.4960842e-05, 6.2500753e-06, 2.8570810e-06, 3.4796615e-04,
        1.0335009e-04, 2.6391530e-07],
       [9.9951851e-01, 4.6972211e-08, 5.4614491e-05, 4.1936628e-06,
        8.3744487e-07, 1.0186117e-05, 2.0517473e-04, 7.9880527e-05,
        8.7462087e-07, 1.2577689e-04],
       [1.4571742e-06, 4.9718238e-09, 1.5038801e-06, 1.2369765e-08,
        9.9835819e-01, 2.0660856e-07, 1.1225045e-06, 2.6037727e-04,
        1.5591773e-06, 1.3755915e-03]], dtype=float32)>

### Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the <a href="https://www.tensorflow.org/guide/keras/overview">Keras API</a>.

For more examples of using Keras, check out the <a href="https://www.tensorflow.org/tutorials/keras/">tutorials</a>. To learn more about building models with Keras, read the <a href="https://www.tensorflow.org/guide/keras">guides<a/>. If you want learn more about loading and preparing data, see the tutorials on <a href="https://www.tensorflow.org/tutorials/load_data/images">image data loading</a> or <a href="https://www.tensorflow.org/tutorials/load_data/csv">CSV data loading</a>.